<a href="https://colab.research.google.com/github/davanstrien/flyswot-gym/blob/master/flyswot_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flyswot_gym import core
from datasets import load_dataset

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
!git config --global credential.helper store

In [5]:
from huggingface_hub import notebook_login

In [6]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [9]:
model_checkpoint = "davanstrien/vit-base-patch16-224-in21k-base-manuscripts"

In [10]:
ds = load_dataset("davanstrien/flysheet", use_auth_token=True, streaming=False, split='train')

Downloading:   0%|          | 0.00/966 [00:00<?, ?B/s]

Using custom data configuration davanstrien--flysheet-34dd57989e10b3fa


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/davanstrien--flysheet-34dd57989e10b3fa/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


In [ ]:
trainer = core.train_model("davanstrien/flysheet", model_checkpoint, 1,"test")

Using custom data configuration davanstrien--flysheet-34dd57989e10b3fa
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/davanstrien--flysheet-34dd57989e10b3fa/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Preparing dataset...

dropping duplicates...

Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/davanstrien--flysheet-34dd57989e10b3fa/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-e88910c5f39df8be.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/davanstrien--flysheet-34dd57989e10b3fa/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-1f9fffbda0342f0c.arrow


getting ID...

...

valid has 256 examples

test has 111 examples

Some weights of the model checkpoint at davanstrien/vit-base-patch16-224-in21k-base-manuscripts were not used when initializing ViTForImageClassification: ['decoder.0.weight', 'vit.embeddings.mask_token', 'decoder.0.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at davanstrien/vit-base-patch16-224-in21k-base-manuscripts and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 256
  Batch size = 4


In [ ]:
trainer